In [2]:
# These are standard modules
import numpy as np
import sympy as sym
from scipy import linalg
from IPython.display import display, Latex, Markdown
np.set_printoptions(suppress=True)


In [3]:
#
# VARIABLES
#

# Time
t = sym.Symbol('t', real=True)

# Components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z', real=True)

# Yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi', real=True)

# Components of linear velocity in the body frame (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z', real=True)

# Components of angular velocity in the body frame (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z', real=True)

# Elevon angles
delta_r, delta_l = sym.symbols('delta_r, delta_l', real=True)

#
# PARAMETERS
#

# Aerodynamic parameters
rho, S, c, b = sym.symbols('rho, S, c, b', real=True)
C_L_0, C_L_alpha, C_L_q, C_L_delta_e = sym.symbols('C_L_0, C_L_alpha, C_L_q, C_L_delta_e', real=True)
C_D_0, C_D_alpha, C_D_q, C_D_delta_e = sym.symbols('C_D_0, C_D_alpha, C_D_q, C_D_delta_e', real=True)
C_m_0, C_m_alpha, C_m_q, C_m_delta_e = sym.symbols('C_m_0, C_m_alpha, C_m_q, C_m_delta_e', real=True)
C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a = sym.symbols('C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a', real=True)
C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a = sym.symbols('C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a', real=True)
C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a = sym.symbols('C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a', real=True)
e, alpha_0, C_D_p, M = sym.symbols('e, alpha_0, C_D_p, M', real=True)
k, k_e = sym.symbols('k, k_e', real=True)

# Mass and inertia parameters
J_x, J_y, J_z, J_xz = sym.symbols('J_x, J_y, J_z, J_xz', real=True)
m, g = sym.symbols('m, g', real=True)

In [4]:
params = {
    g: 9.81,               # Gravity (m/s²)
    m: 1.56,               # Mass of the UAV (kg)
    J_x: 0.1147,           # Moment of inertia about x-axis (kg·m²) [UPDATED 02/28/2025]
    J_y: 0.0576,           # Moment of inertia about y-axis (kg·m²) [UPDATED 02/28/2025]
    J_z: 0.1712,           # Moment of inertia about z-axis (kg·m²) [UPDATED 02/28/2025]
    J_xz: 0.0015,          # Product of inertia (kg·m²)             [UPDATED 02/28/2025]

    S: 0.4696,             # Wing area (m²)
    b: 1.4224,             # Wingspan (m)
    c: 0.3302,             # Mean aerodynamic chord (m)

    rho: 1.2682,           # Air density (kg/m³)

    # Lift Coefficients
    C_L_0: 0.2,            # Lift coefficient at zero AoA
    C_L_alpha: 4.8,        # Lift curve slope (1/rad)
    C_L_q: 2.2,            # Pitch rate effect on lift (1/rad)

    # Drag Coefficients
    C_D_0: 0.02,           # Zero-lift drag coefficient
    C_D_alpha: 0.30,       # Drag change per AoA (1/rad)
    C_D_q: 0.0,            # Pitch rate effect on drag (1/rad)
    C_D_p: 0.03,           # Parasitic drag coefficient

    # Pitching Moment Coefficients
    C_m_0: -0.02,          # Pitching moment at zero AoA
    C_m_alpha: -0.6,       # Pitching moment change per AoA (1/rad)
    C_m_q: -1.8,           # Pitch rate effect on moment (1/rad)
    C_m_delta_e: -0.35,    # Effect of elevator deflection on pitching moment (1/rad)

    # Side Force Coefficients
    C_Y_0: 0.0,            # Side force at zero sideslip
    C_Y_beta: -0.08,       # Side force per sideslip angle (1/rad)
    C_Y_p: 0.0,            # Side force due to roll rate
    C_Y_r: 0.0,            # Side force due to yaw rate
    C_Y_delta_a: 0.0,      # Side force due to aileron deflection

    # Roll Moment Coefficients
    C_l_0: 0.0,            # Roll moment at zero sideslip
    C_l_beta: -0.10,       # Roll moment due to sideslip (1/rad)
    C_l_p: -0.45,          # Roll damping derivative (1/rad)
    C_l_r: 0.03,           # Roll moment due to yaw rate (1/rad)
    C_l_delta_a: 0.18,     # Aileron effect on roll (1/rad)

    # Yaw Moment Coefficients
    C_n_0: 0.0,            # Yaw moment at zero sideslip
    C_n_beta: 0.008,       # Yaw moment due to sideslip (1/rad)
    C_n_p: -0.022,         # Yaw moment due to roll rate (1/rad)
    C_n_r: -0.009,         # Yaw damping derivative (1/rad)
    C_n_delta_a: -0.004,   # Aileron effect on yaw (1/rad)

    # Control Derivatives
    C_L_delta_e: 0.30,     # Effect of elevator deflection on lift (1/rad)
    C_D_delta_e: 0.32,     # Effect of elevator deflection on drag (1/rad)

    # Efficiency Factors
    e: 0.85,               # Oswald efficiency factor
    alpha_0: 0.45,         # Zero-lift angle of attack (rad)

    # Additional Drag & Lift Coefficients
    M: 50.0,               # Sigmoid blending function parameter
    k_e: 0.01,             # Drag due to elevator deflection (empirical coefficient)
    k: 0.048               # Induced drag factor
}

In [5]:
# Get airspeed, angle of attack, and angle of sideslip
V_a = sym.sqrt(v_x**2 + v_y**2 + v_z**2)
alpha = sym.atan(v_z / v_x)
beta = sym.asin(v_y / V_a)

# Convert from right and left elevon deflections to equivalent elevator and aileron deflections
delta_e = (delta_r + delta_l) / 2
delta_a = (-delta_r + delta_l) / 2

# Longitudinal aerodynamics
C_L = C_L_0 + C_L_alpha * alpha
F_lift = rho * V_a**2 * S * (C_L + C_L_q * (c / (2 * V_a)) * w_y + C_L_delta_e * delta_e) / 2
F_drag = rho * V_a**2 * S * ((C_D_0 + k * C_L**2) + C_D_q * (c / (2 * V_a)) * w_y + k_e * (C_L_delta_e * delta_e)**2) / 2
f_x, f_z = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha)], [sym.sin(alpha), sym.cos(alpha)]]) @ sym.Matrix([[-F_drag], [-F_lift]])
tau_y = rho * V_a**2 * S * c * (C_m_0 + C_m_alpha * alpha + C_m_q * (c / (2 * V_a)) * w_y + C_m_delta_e * delta_e) / 2

# Lateral aerodynamics
f_y =   rho * V_a**2 * S *     (C_Y_0 + C_Y_beta * beta + C_Y_p * (b / (2 * V_a)) * w_x + C_Y_r * (b / (2 * V_a)) * w_z + C_Y_delta_a * delta_a) / 2
tau_x = rho * V_a**2 * S * b * (C_l_0 + C_l_beta * beta + C_l_p * (b / (2 * V_a)) * w_x + C_l_r * (b / (2 * V_a)) * w_z + C_l_delta_a * delta_a) / 2
tau_z = rho * V_a**2 * S * b * (C_n_0 + C_n_beta * beta + C_n_p * (b / (2 * V_a)) * w_x + C_n_r * (b / (2 * V_a)) * w_z + C_n_delta_a * delta_a) / 2

In [6]:
v_inB_ofWB = sym.Matrix([v_x, v_y, v_z])
w_inB_ofWB = sym.Matrix([w_x, w_y, w_z])

In [7]:
J_inB = sym.Matrix([[  J_x,    0, -J_xz],
                    [    0,  J_y,     0],
                    [-J_xz,    0,   J_z]])

In [8]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

In [9]:
R_inW_ofB = Rz @ Ry @ Rx

In [10]:
# First, compute the inverse of N
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

# Then, take the inverse of this result to compute N
N = sym.simplify(Ninv.inv())

In [11]:
# Total force
f_inB = R_inW_ofB.T * sym.Matrix([0, 0, m * g]) + sym.Matrix([f_x, f_y, f_z])

# Total torque
tau_inB = sym.Matrix([tau_x, tau_y, tau_z])

# Displaying the UAV Equations of Motion

In this cell we display the full UAV equations of motion. The state vector is assumed to be

$$
x = \begin{bmatrix} p_x \\ p_y \\ p_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \end{bmatrix},
$$

and the control inputs are given by

$$
u = \begin{bmatrix} \delta_r \\ \delta_l \end{bmatrix}.
$$

The dynamics have been derived symbolically (including the kinematic transformations and force/moment balances) as

$$
\dot{x} = f(x,u) = \begin{bmatrix} R_{W}^{B}\,v_B \\ N\,w_B \\ \frac{1}{m}\Big(f_{inB} - w_B \times (m\,v_B)\Big) \\ J_B^{-1}\Big(\tau_{inB} - w_B \times (J_B\,w_B)\Big) \end{bmatrix}.
$$

Below, we substitute the parameter values and display the equations in LaTeX.


In [12]:
# %% [code]
# Compute the full UAV equations of motion symbolically
f_sym = sym.Matrix.vstack(
    R_inW_ofB * v_inB_ofWB,                                   # Kinematics: time derivative of position
    N * w_inB_ofWB,                                           # Kinematics: Euler angle rates
    (1 / m) * (f_inB - w_inB_ofWB.cross(m * v_inB_ofWB)),     # Translational dynamics
    J_inB.inv() * (tau_inB - w_inB_ofWB.cross(J_inB * w_inB_ofWB))  # Rotational dynamics
)

# Substitute the numerical parameters into the equations
f_uav = sym.simplify(f_sym.subs(params))

# # Define LaTeX strings for the state and input vectors
mdot_str = r'\begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix}'
m_str = r'\begin{bmatrix} p_x \\ p_y \\ p_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \end{bmatrix}'
n_str = r'\begin{bmatrix} \delta_r \\ \delta_l \end{bmatrix}'

# # Display the equations of motion using Markdown with LaTeX formatting
display(Markdown(fr'$$\LARGE {mdot_str} = f\left( {m_str}, {n_str} \right) = {sym.latex(f_uav)}$$'))


$$\LARGE \begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix} = f\left( \begin{bmatrix} p_x \\ p_y \\ p_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \end{bmatrix}, \begin{bmatrix} \delta_r \\ \delta_l \end{bmatrix} \right) = \left[\begin{matrix}v_{x} \cos{\left(\psi \right)} \cos{\left(\theta \right)} + v_{y} \left(\sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - \sin{\left(\psi \right)} \cos{\left(\phi \right)}\right) + v_{z} \left(\sin{\left(\phi \right)} \sin{\left(\psi \right)} + \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)}\right)\\v_{x} \sin{\left(\psi \right)} \cos{\left(\theta \right)} + v_{y} \left(\sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} + \cos{\left(\phi \right)} \cos{\left(\psi \right)}\right) - v_{z} \left(\sin{\left(\phi \right)} \cos{\left(\psi \right)} - \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)}\right)\\- v_{x} \sin{\left(\theta \right)} + v_{y} \sin{\left(\phi \right)} \cos{\left(\theta \right)} + v_{z} \cos{\left(\phi \right)} \cos{\left(\theta \right)}\\\frac{w_{y} \sin{\left(\phi \right)} + w_{z} \cos{\left(\phi \right)}}{\cos{\left(\theta \right)}}\\w_{y} \cos{\left(\phi \right)} - w_{z} \sin{\left(\phi \right)}\\w_{x} + w_{y} \sin{\left(\phi \right)} \tan{\left(\theta \right)} + w_{z} \cos{\left(\phi \right)} \tan{\left(\theta \right)}\\\frac{- 0.190880358974359 v_{x} \sqrt{v_{x}^{2} + v_{z}^{2}} \left(v_{x}^{2} + v_{y}^{2} + v_{z}^{2}\right) \left(0.000225 \left(\delta_{l} + \delta_{r}\right)^{2} + 1.10592 \left(\operatorname{atan}{\left(\frac{v_{z}}{v_{x}} \right)} + 0.0416666666666667\right)^{2} + 0.02\right) \left|{v_{x}}\right| + v_{x} \left(v_{x}^{2} + v_{z}^{2}\right) \left(1.0 v_{y} w_{z} - 1.0 v_{z} w_{y} - 9.81 \sin{\left(\theta \right)}\right) + 0.190880358974359 v_{z} \left(0.36322 w_{y} + \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}} \left(0.15 \delta_{l} + 0.15 \delta_{r} + 4.8 \operatorname{atan}{\left(\frac{v_{z}}{v_{x}} \right)} + 0.2\right)\right) \sqrt{v_{x}^{4} + v_{x}^{2} v_{y}^{2} + 2 v_{x}^{2} v_{z}^{2} + v_{y}^{2} v_{z}^{2} + v_{z}^{4}} \left|{v_{x}}\right|}{v_{x} \left(v_{x}^{2} + v_{z}^{2}\right)}\\- 1.0 v_{x} w_{z} + 1.0 v_{z} w_{x} - 0.0152704287179487 \left(v_{x}^{2} + v_{y}^{2} + v_{z}^{2}\right) \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} + 9.81 \sin{\left(\phi \right)} \cos{\left(\theta \right)}\\\frac{v_{x} \left(v_{x}^{2} + v_{z}^{2}\right) \left(1.0 v_{x} w_{y} - 1.0 v_{y} w_{x} + 9.81 \cos{\left(\phi \right)} \cos{\left(\theta \right)}\right) - 0.190880358974359 v_{x} \left(0.36322 w_{y} + \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}} \left(0.15 \delta_{l} + 0.15 \delta_{r} + 4.8 \operatorname{atan}{\left(\frac{v_{z}}{v_{x}} \right)} + 0.2\right)\right) \sqrt{v_{x}^{4} + v_{x}^{2} v_{y}^{2} + 2 v_{x}^{2} v_{z}^{2} + v_{y}^{2} v_{z}^{2} + v_{z}^{4}} \left|{v_{x}}\right| - 0.190880358974359 v_{z} \sqrt{v_{x}^{2} + v_{z}^{2}} \left(v_{x}^{2} + v_{y}^{2} + v_{z}^{2}\right) \left(0.000225 \left(\delta_{l} + \delta_{r}\right)^{2} + 1.10592 \left(\operatorname{atan}{\left(\frac{v_{z}}{v_{x}} \right)} + 0.0416666666666667\right)^{2} + 0.02\right) \left|{v_{x}}\right|}{v_{x} \left(v_{x}^{2} + v_{z}^{2}\right)}\\0.332316476549662 \delta_{l} v_{x}^{2} + 0.332316476549662 \delta_{l} v_{y}^{2} + 0.332316476549662 \delta_{l} v_{z}^{2} - 0.332316476549662 \delta_{r} v_{x}^{2} - 0.332316476549662 \delta_{r} v_{y}^{2} - 0.332316476549662 \delta_{r} v_{z}^{2} - 0.369053572269498 v_{x}^{2} \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} - 0.369053572269498 v_{y}^{2} \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} - 0.369053572269498 v_{z}^{2} \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} + 0.0174413363491303 w_{x} w_{y} - 1.18245380651127 w_{x} \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}} - 0.990637855314069 w_{y} w_{z} + 0.0785893843999064 w_{z} \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}\\- 17.3611111111111 w_{x} \left(0.0015 w_{x} - 0.1712 w_{z}\right) - 17.3611111111111 w_{z} \left(0.1147 w_{x} - 0.0015 w_{z}\right) - \left(0.50729432653835 w_{y} + \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}} \left(0.298729750131944 \delta_{l} + 0.298729750131944 \delta_{r} + 1.02421628616667 \operatorname{atan}{\left(\frac{v_{z}}{v_{x}} \right)} + 0.0341405428722222\right)\right) \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}\\- 0.00203639567583824 \delta_{l} v_{x}^{2} - 0.00203639567583824 \delta_{l} v_{y}^{2} - 0.00203639567583824 \delta_{l} v_{z}^{2} + 0.00203639567583824 \delta_{r} v_{x}^{2} + 0.00203639567583824 \delta_{r} v_{y}^{2} + 0.00203639567583824 \delta_{r} v_{z}^{2} + 0.0165586580590406 v_{x}^{2} \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} + 0.0165586580590406 v_{y}^{2} \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} + 0.0165586580590406 v_{z}^{2} \operatorname{asin}{\left(\frac{v_{y}}{\sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}} \right)} + 0.333680852830162 w_{x} w_{y} - 0.0490698461814857 w_{x} \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}} - 0.0174413363491303 w_{y} w_{z} - 0.01514715455696 w_{z} \sqrt{v_{x}^{2} + v_{y}^{2} + v_{z}^{2}}\end{matrix}\right]$$

# Create Numerical Function for the UAV Equations of Motion

To evaluate the UAV dynamics numerically, we convert our symbolic function $f_{uav}$ (which represents the full equations of motion) into a numerical lambda function using `sym.lambdify`. In our UAV model, the state vector is

$$
x = \begin{bmatrix} p_x,\, p_y,\, p_z,\, \psi,\, \theta,\, \phi,\, v_x,\, v_y,\, v_z,\, w_x,\, w_y,\, w_z \end{bmatrix}^T,
$$

and the control input vector is

$$
u = \begin{bmatrix} \delta_r,\, \delta_l \end{bmatrix}^T.
$$

The following code creates a lambda function that accepts these 12 state variables and 2 control inputs (a total of 14 arguments) and returns the state derivative $\\dot{x}$.


In [13]:
# Create a numerical function for the UAV equations of motion.
# This lambda function takes 12 state variables and 2 control inputs as arguments
# and returns the computed state derivative (x_dot) as a NumPy array.

f_uav_reduced = f_uav[[1, 3, 4,5,6,7,8,9,10,11], :]


f_uav_num_reduced = sym.lambdify(
    [p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l],
    f_uav_reduced
)

In [14]:
from scipy.optimize import minimize

In [15]:
f_num_for_minimize = lambda x: np.linalg.norm(f_uav_num_reduced(*x).flatten())**2

In [ ]:
# Initial guess
x0 = [
    0, # <-- guess at p_y_e
    0, # <-- guess at psi_e
    0, # <-- guess at theta_e
    0, # <-- guess at phi_e
    2.5, # <-- guess at v_x_e - ESTIMATED MANUALLY BY PERFORMING NUMEROUS ITERATIONS
    0, # <-- guess at v_y_e
    0.15, # <-- guess at v_z_e - ESTIMATED MANUALLY BY PERFORMING NUMEROUS ITERATIONS
    0, # <-- guess at w_x_e
    0, # <-- guess at w_y_e
    0, # <-- guess at w_z_e
    0, # <-- guess at delta_r_e
    0, # <-- guess at delta_l_e

]

# Find minimum ("tol" is a tolerance that says how accurate you want the solution)
sol = minimize(f_num_for_minimize, x0, tol=1e-4)

# Show solution
print(sol)

# Extract equilibrium state variables from the solution vector
(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e) = sol.x

# Print the equilibrium state values with appropriate formatting:
print(f'  p_y_e   = {p_y_e:7.3f}')
print(f'  psi_e   = {psi_e:7.3f}')
print(f'  theta_e = {theta_e:7.3f}')
print(f'  phi_e   = {phi_e:7.3f}')
print(f'  v_x_e   = {v_x_e:7.3f}')
print(f'  v_y_e   = {v_y_e:7.3f}')
print(f'  v_z_e   = {v_z_e:7.3f}')
print(f'  w_x_e   = {w_x_e:7.3f}')
print(f'  w_y_e   = {w_y_e:7.3f}')
print(f'  w_z_e   = {w_z_e:7.3f}')
print(f'  delta_r_e   = {delta_r_e:7.3f}')
print(f'  delta_l_e   = {delta_l_e:7.3f}')


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.3380849700007867e-10
        x: [ 0.000e+00 -3.389e-05  4.893e-01  3.415e-06  3.140e+00
             1.435e-04  2.538e+00 -3.639e-06 -1.962e-07  2.883e-06
            -1.223e+00 -1.223e+00]
      nit: 29
      jac: [ 0.000e+00  4.053e-06 -3.975e-06  4.097e-05 -1.601e-05
            -2.009e-06 -6.525e-05 -7.460e-06  4.091e-05 -1.023e-05
            -1.697e-05  1.028e-05]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  8.684e+00 ...  4.965e+00 -4.947e+00]
            ...
            [ 0.000e+00  4.965e+00 ...  3.034e+00 -2.733e+00]
            [ 0.000e+00 -4.947e+00 ... -2.733e+00  3.023e+00]]
     nfev: 533
     njev: 41
  p_y_e   =   0.000
  psi_e   =  -0.000
  theta_e =   0.489
  phi_e   =   0.000
  v_x_e   =   3.140
  v_y_e   =   0.000
  v_z_e   =   2.538
  w_x_e   =  -0.000
  w_y_e   =  -0.000
  w_z_e   =   0.000
  delta_r_e   =  -1.223
  delta_l_e   =  -

In [89]:
# Find A and B in symbolic form
A_sym = f_uav_reduced.jacobian([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z])
B_sym = f_uav_reduced.jacobian([delta_r, delta_l])

# Create lambda functions to allow numerical evaluation of A and B
A_num = sym.lambdify([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l], A_sym)
B_num = sym.lambdify([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l], B_sym)

# Find A and B in numeric form (making sure the result is floating-point)
A = A_num(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e).astype(float)
B = B_num(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e).astype(float)

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

A =
[[ 0.          3.96406623 -0.00002591 -2.53788912 -0.00002992  1.
  -0.00001935  0.          0.          0.        ]
 [ 0.          0.          0.00000174 -0.00000022  0.          0.
   0.          0.          0.00000387  1.13295783]
 [ 0.          0.          0.         -0.00000288  0.          0.
   0.          0.          1.         -0.00000342]
 [ 0.          0.          0.0000037  -0.0000001   0.          0.
   0.          1.          0.00000182  0.53253492]
 [ 0.          0.         -8.65875125  0.         -0.43236308  0.00008409
   4.1686687   0.         -2.3619333   0.00014351]
 [ 0.          0.         -0.00001575  8.65875125 -0.00000459 -0.06164792
  -0.00000502  2.53788912  0.         -3.13960529]
 [ 0.          0.         -4.61108737 -0.00002957 -1.75969671 -0.00014885
  -4.64667217 -0.00014351  2.92193154  0.        ]
 [ 0.          0.          0.          0.          0.00003817 -1.48989823
   0.00003086 -4.7736588  -0.00000292  0.31727171]
 [ 0.          0.          0

LQR Time!

In [90]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K, P

Q = np.diag([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
R = np.diag([1., 1.])

K, P = lqr(A, B, Q, R)
print(f'K =\n{K.tolist()}\n')

print(f'p = {linalg.eigvals(A - B @ K)}')

K =
[[-0.7071506235443417, -7.932827968413466, -2.801042612310188, -3.91166626240799, 0.11486972020887214, -1.3081477754663797, 0.33803156914717364, -0.9050620962457437, -0.7646821760400226, -8.171570519042717], [0.7070629361102652, 7.932369857155983, -2.80102162357891, 3.9120229863339597, 0.1148675799683302, 1.3081104871799247, 0.3380144643275086, 0.905064382495035, -0.7646766121153041, 8.172592976158896]]

p = [-8.38253254+0.j         -5.84989292+3.810451j   -5.84989292-3.810451j
 -2.2375583 +1.96384395j -2.2375583 -1.96384395j -0.10898205+0.44937737j
 -0.10898205-0.44937737j -1.08221101+0.j         -1.34867101+2.73554365j
 -1.34867101-2.73554365j]
